In [497]:
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np
lone_pine = pd.read_csv(r'C:\Users\wells\OneDrive\Desktop\lone_pine.csv')
crsp_data = pd.read_csv('StockPriceMonthly.csv', encoding='utf8', low_memory=False)

# pre-processing

In [498]:
lone_pine['Ticker'] = lone_pine['Ticker'].astype('str')
def ab(aa):
    if '.' in aa:
        return aa.split('.')[0]
    if '/' in aa:
        return aa.split('/')[0]
    else:
        return aa
lone_pine['Ticker'] = lone_pine['Ticker'].apply(ab)

In [499]:
def abc(aa):
    aa = aa.split('/')
    year = aa[2]
    month = aa[0]
    day = aa[1]
    return year+month
lone_pine['Date'] = lone_pine['Date'].apply(abc)

In [500]:
def abcd(aa):
    return str(aa)[0:6]
crsp_data['date'] = crsp_data['date'].apply(abcd)

In [501]:
add_df = pd.DataFrame(columns=lone_pine.columns)
for index in tqdm_notebook(range(len(lone_pine))):
    each_data = lone_pine.loc[index]
    date = int(each_data['Date'])
    if date % 4 == 0:
        date = date - 12 + 100
    each_data['Date'] = str(date+1)
    each_data['Change'] = 0
    each_data['Change.1'] = 0
    add_df.loc[-1] = list(each_data)
    add_df = add_df.reset_index(drop=True)
    date = int(each_data['Date'])
    each_data['Date'] = str(date+1)
    each_data['Change'] = 0
    each_data['Change.1'] = 0
    add_df.loc[-1] = list(each_data)
    add_df = add_df.reset_index(drop=True)
lone_pine_a = pd.concat([lone_pine, add_df], ignore_index=True)

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\wells\AppData\Local\Continuu

In [502]:
def class_(df_col):
    if df_col == 'CL A':
        return 'A'
    elif df_col == 'CL B':
        return 'B'
    elif df_col == 'CL C':
        return 'C'
    elif df_col == 'CL D':
        return 'D'
    else:
        return np.nan
lone_pine_a['Class'] = lone_pine_a['Class'].apply(class_)

In [503]:
def division(df):
    if df['Shares'] == 0:
        return 0
    else:
        return df['Value'] / df['Shares']
lone_pine_a['price'] = lone_pine_a.apply(division, axis=1)

In [504]:
crsp_data.columns = ['PERMNO', 'Date', 'Ticker', 'COMNAM', 'Class', 'DIVAMT', 'PRC', 'VOL']

In [505]:
# def class_1(df_col):
#     if df_col == 'A':
#         return 'A'
#     elif df_col == 'B':
#         return 'B'
#     elif df_col == 'C':
#         return 'C'
#     elif df_col == 'D':
#         return 'D'
#     else:
#         return 0
# crsp_data['Class'] = crsp_data['Class'].apply(class_1)

In [506]:
def to_abs(df_col):
    return abs(df_col)
crsp_data_c = crsp_data[['Date', 'Ticker', 'PRC', 'Class']]
crsp_data_c = crsp_data_c[crsp_data_c['PRC'].notnull()].drop_duplicates(keep='first')
crsp_data_c['PRC'] = crsp_data_c['PRC'].apply(to_abs)

In [507]:
lone_pine_all = lone_pine_a.merge(crsp_data_c, on=['Date', 'Ticker'], how = 'left')

In [508]:
lone_pine_all.drop(['% Port', '% OS', 'Hist'], axis=1, inplace=True)

In [509]:
for name in list(set(list(lone_pine_all['Name']))):
    if len(lone_pine_a[lone_pine_a['Name']==name].reset_index(drop=True)) != len(lone_pine_all[lone_pine_all['Name']==name].reset_index(drop=True)):
        drop_df = lone_pine_all[lone_pine_all['Name']==name].loc[lone_pine_all['Class_x']!=lone_pine_all['Class_y']]
        lone_pine_all.drop(list(drop_df.index), inplace=True)

In [510]:
lone_pine_all = lone_pine_all.sort_values(by='Date').reset_index(drop=True)
df_list = []
for each in tqdm_notebook(lone_pine_all.groupby(by='Name')):
    name, df = each
    df.loc[:, 'last_price'] = df['price'].shift()
    df.loc[:, 'last_PRC'] = df['PRC'].shift()
    df_list.append(df)

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [511]:
lone_pine_all = pd.concat(df_list)

In [512]:
error = lone_pine_all[lone_pine_all['PRC'].isnull()][['Name', 'Ticker', 'PRC', 'Date']].drop_duplicates(keep='first', inplace=False)

In [514]:
test = lone_pine_all
new_error = []
for index in tqdm_notebook(error.index):
    each = error.loc[index]
    df = crsp_data[crsp_data['Ticker']==each['Ticker']]
    try:
        if int(each['Date']) > int(list(df['Date'])[-1]):
            test = test.drop(index)
    except:
        new_error.append(list(each))
lone_pine_all = test

In [519]:
pd.DataFrame(new_error).to_csv('error.csv', index=False)

# revenue

In [536]:
lone_pine_all_t = lone_pine_all
for index in tqdm_notebook(lone_pine_all.index):
    each_data = lone_pine_all.loc[index]
    if each_data['Shares'] == 0 and each_data['Change'] == 0 and each_data['Value'] == 0 and each_data['Change.1'] == 0:
        lone_pine_all_t.drop(index, inplace=True)
lone_pine_all = lone_pine_all_t

In [571]:
def prc_price(df):
    if np.isnan(df['PRC']):
        return df['price']
    else:
        return df['PRC']
lone_pine_all['PRC'] = lone_pine_all.apply(prc_price, axis=1)

In [574]:
def prc_price(df):
    if np.isnan(df['last_PRC']):
        return df['last_price']
    else:
        return df['last_PRC']
lone_pine_all['last_PRC'] = lone_pine_all.apply(prc_price, axis=1)

In [577]:
lone_pine_all.drop(['Class_x', 'price', 'last_price', 'price'], axis=1, inplace=True)
lone_pine_all.columns = ['Name', 'Ticker', 'Shares', 'Shares_Change', 'Value', 'Value_Change', 'Date', 'Price',
                         'Class', 'Last_Price']

In [1088]:
new_share = lone_pine_all[lone_pine_all['Shares']==lone_pine_all['Shares_Change']]
new_share['New_Value'] = new_share['Shares'] * new_share['Price']

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1089]:
new_share.to_csv('new_share.csv', index=False)

In [1090]:
sell_share = lone_pine_all[lone_pine_all['Shares']==0]
sell_share['New_Value'] = sell_share['Shares_Change'] * sell_share['Price']

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1091]:
sell_share.to_csv('sell_share.csv', index=False)

In [1092]:
more_share = lone_pine_all[lone_pine_all['Shares_Change']>0].loc[lone_pine_all['Shares']!=lone_pine_all['Shares_Change']]
more_share['New_Value'] = more_share['Shares'] * more_share['Price']

In [1093]:
more_share.to_csv('more_share.csv', index=False)

In [1094]:
less_share = lone_pine_all[lone_pine_all['Shares_Change']<0].loc[lone_pine_all['Shares']!=0]
less_share['New_Value'] = (less_share['Shares'] + abs(less_share['Shares_Change'])) * less_share['Price']

In [1095]:
less_share.to_csv('less_share.csv', index=False)

In [1105]:
same_share = lone_pine_all[lone_pine_all['Shares_Change']==0]
same_share['New_Value'] = same_share['Shares'] * same_share['Price']

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1106]:
same_share.to_csv('same_share.csv', index=False)

In [1243]:
total_asset = []
for date in list(set(lone_pine_all['Date'])):
    lone_pine_all_t = lone_pine_all[lone_pine_all['Date']==date]
    total_asset.append(sum(lone_pine_all_t['New_Value']))

In [1246]:
revenue = []
for date in list(set(lone_pine_all['Date'])):
    lone_pine_all_t = lone_pine_all[lone_pine_all['Date']==date]

    sell_shares = lone_pine_all_t[lone_pine_all_t['Shares']==0]
    more_shares = lone_pine_all_t[lone_pine_all_t['Shares_Change']>0].loc[lone_pine_all_t['Shares']!=lone_pine_all_t['Shares_Change']]
    less_shares = lone_pine_all_t[lone_pine_all_t['Shares_Change']<0].loc[lone_pine_all_t['Shares']!=0]
    same_shares = lone_pine_all_t[lone_pine_all_t['Shares_Change']==0]

    sell_shares['diff'] = abs(sell_shares['Shares_Change']) * (sell_shares['Price'] - sell_shares['Last_Price'])
    more_shares['diff'] = (more_shares['Shares'] - more_shares['Shares_Change']) * (more_shares['Price'] - more_shares['Last_Price'])
    less_shares['diff'] = (less_shares['Shares'] + abs(less_shares['Shares_Change'])) * (less_shares['Price'] - less_shares['Last_Price'])
    same_shares['diff'] = same_shares['Shares'] * (same_shares['Price'] - same_shares['Last_Price'])
    
    sell_shares = sell_shares[sell_shares['diff'].notnull()]
    more_shares = more_shares[more_shares['diff'].notnull()]
    less_shares = less_shares[less_shares['diff'].notnull()]
    same_shares = same_shares[same_shares['diff'].notnull()]
    
    revenue.append(sum(sell_shares['diff']) + sum(more_shares['diff']) + sum(less_shares['diff']) + sum(same_shares['diff']))

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [1247]:
revenue_df = pd.DataFrame({'date':list(set(lone_pine_all['Date'])),
                          'asset':total_asset,
                          'revenue':revenue})

In [1250]:
revenue_df = revenue_df.sort_values(by='date').reset_index(drop=True)

In [1252]:
revenue_df['last_asset'] = revenue_df['asset'].shift()

In [1256]:
revenue_df['return_rate'] = round(revenue_df['revenue'] / revenue_df['last_asset'] * 100, 5)

In [1258]:
fama = pd.read_csv('fama.csv')
fama['date'] = fama['date'].astype('str')

fama_3 = fama.merge(revenue_df[['date', 'return_rate']], on='date', how='left')

fama_3 = fama_3[fama_3['return_rate'].notnull()]

fama_3['return_rate'] = fama_3['return_rate'] - fama_3['RF']

In [1259]:
fama = pd.read_csv('fama5.csv')
fama['date'] = fama['date'].astype('str')

fama_5 = fama.merge(revenue_df[['date', 'return_rate']], on='date', how='left')

fama_5 = fama_5[fama_5['return_rate'].notnull()]

fama_5['return_rate'] = fama_5['return_rate'] - fama_5['RF']

# lasso

In [1155]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [1302]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1303]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'train_score', 'test_score', 'e', 'coeff_used'])
alpha = 5
while alpha > 0.001:
    lasso = Lasso(alpha)
    lasso.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = lasso.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = lasso.predict(x_test_std)
    score = r2_score(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(lasso.coef_!=0)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    alpha_df.loc[-1] = [alpha] + list(lasso.coef_) + [score_train, score, e, coeff_used]
    alpha_df = alpha_df.reset_index(drop=True)
    alpha = alpha - 0.001

In [1304]:
print('train_score:',max(alpha_df['train_score']))
print('test_score:',max(alpha_df['test_score']))
alpha_df.sort_values(by='test_score', ascending=False)[0:6]

train_score: 0.012488949939432725
test_score: -13.421032393251707


,alpha,coef1,coef2,coef3,train_score,test_score,e,coeff_used
0,5.000,-3.324834,17.456317,11.435653,0.010968,-13.421032,444.592189,3.0
1,4.999,-3.326621,17.457768,11.436771,0.010968,-13.421664,444.611653,3.0
2,4.998,-3.328407,17.459218,11.437888,0.010969,-13.422295,444.631123,3.0
3,4.997,-3.330193,17.460669,11.439006,0.010970,-13.422927,444.650599,3.0
4,4.996,-3.331979,17.462120,11.440123,0.010970,-13.423559,444.670081,3.0
5,4.995,-3.333765,17.463571,11.441241,0.010971,-13.424191,444.689570,3.0


In [1305]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [ ]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'coef4', 'coef5', 'train_score', 'test_score', 'e', 'coeff_used'])
alpha = 5
while alpha > 0.001:
    lasso = Lasso(alpha)
    lasso.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = lasso.predict(x_train_std)
    score_train=r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = lasso.predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(lasso.coef_!=0)
    alpha_df.loc[-1] = [alpha] + list(lasso.coef_) + [score_train, score, e, coeff_used]
    alpha_df = alpha_df.reset_index(drop=True)
    alpha = alpha - 0.001

In [ ]:
print('train_score:',max(alpha_df['train_score']))
print('test_score:',max(alpha_df['test_score']))
alpha_df.sort_values(by='test_score', ascending=False)[0:6]

# Ridge

In [847]:
from sklearn.linear_model import Ridge

In [1290]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1291]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'train_score', 'test_score', 'e', 'coeff_used'])
alpha = 5
while alpha >= 0.001:
    rr = Ridge(alpha)
    rr.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = rr.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = rr.predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(rr.coef_!=0)
    alpha_df.loc[-1] = [alpha] + list(rr.coef_) + [score_train, score, e, coeff_used]
    alpha_df = alpha_df.reset_index(drop=True)
    alpha = alpha - 0.001

In [1292]:
alpha_df.sort_values(by='test_score', ascending=False)[0:6]
print('train_score:',max(alpha_df['train_score']))
print('test_score:',max(alpha_df['test_score']))

train_score: 0.015562905934369708
test_score: -89.9436077742032


In [1293]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1294]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'coef4', 'coef5', 'train_score', 'test_score', 'e', 'coeff_used'])
alpha = 5
while alpha >= 0.001:
    rr = Ridge(alpha)
    rr.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = rr.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = rr.predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(rr.coef_!=0)
    alpha_df.loc[-1] = [alpha] + list(rr.coef_) + [score_train, score, e, coeff_used]
    alpha_df = alpha_df.reset_index(drop=True)
    alpha = alpha - 0.001

In [1295]:
alpha_df.sort_values(by='test_score', ascending=False)[0:6]
print('train_score:',max(alpha_df['train_score']))
print('test_score:',max(alpha_df['test_score']))

train_score: 0.022269273889409247
test_score: -68.72711893122063


# SVR

In [857]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve

In [1296]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1297]:
c_df = pd.DataFrame(columns=['c', 'coef1', 'coef2', 'coef3', 'train_score', 'test_score', 'e', 'coeff_used'])
c = 100
while c > 1:
    svr_lin = SVR(kernel='linear', C=c, gamma='auto')
    svr_lin.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = svr_lin.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = svr_lin.predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(svr_lin.coef_!=0)
    c_df.loc[-1] = [c] + list(svr_lin.coef_[0]) + [score_train, score, e, coeff_used]
    c_df = c_df.reset_index(drop=True)
    c = c - 1

In [1298]:
c_df.sort_values(by='test_score', ascending=False)[0:6]
print('train_score:',max(alpha_df['train_score']))
print('test_score:',max(alpha_df['test_score']))

train_score: 0.022269273889409247
test_score: -68.72711893122063


In [1299]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1300]:
c_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'coef4', 'coef5', 'train_score', 'test_score', 'e', 'coeff_used'])
c = 100
while c > 1:
    svr_lin = SVR(kernel='linear', C=c, gamma='auto')
    svr_lin.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = svr_lin.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = svr_lin.fit(x_train_std, y_train['return_rate'].values).predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(svr_lin.coef_!=0)
    c_df.loc[-1] = [c] + list(svr_lin.coef_[0]) + [score_train, score, e, coeff_used]
    c_df = c_df.reset_index(drop=True)
    c = c - 1

In [1301]:
c_df.sort_values(by='test_score', ascending=False)[0:6]
print('train_score:',max(alpha_df['train_score']))
print('test_score:',max(alpha_df['test_score']))

train_score: 0.022269273889409247
test_score: -68.72711893122063


# LinearRegression

In [892]:
from sklearn.linear_model import LinearRegression

In [1280]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1281]:
lr = LinearRegression(alpha)
lr.fit(x_train_std, y_train['return_rate'].values)
y_predict_train = lr.predict(x_train_std)
score_train = r2_score(y_train['return_rate'].values, y_predict_train)
y_predict = lr.predict(x_test_std)
score=r2_score(y_test['return_rate'].values, y_predict)
e = mean_squared_error(y_test['return_rate'].values, y_predict)
coeff_used = np.sum(lr.coef_!=0)
print('coef:', lr.coef_)
print('score_train:',score_train)
print('score_prediction:',score)
print('e:',e)
# list(lr.coef_) + [score_train, score, e, coeff_used]

coef: [-14.87442408  31.13021482  18.7217049 ]
score_train: 0.01735656040364142
score_prediction: -5.9545065294916135
e: 2850.6897801390965


In [1282]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1283]:
lr = LinearRegression(alpha)
lr.fit(x_train_std, y_train['return_rate'].values)
y_predict_train = lr.predict(x_train_std)
score_train = r2_score(y_train['return_rate'].values, y_predict_train)
y_predict = lr.predict(x_test_std)
score=r2_score(y_test['return_rate'].values, y_predict)
e = mean_squared_error(y_test['return_rate'].values, y_predict)
coeff_used = np.sum(lr.coef_!=0)
print('coef:', lr.coef_)
print('score_train:',score_train)
print('score_prediction:',score)
print('e:',e)
# list(lr.coef_) + [score, e, coeff_used]

coef: [ 5.05150275 -1.69255817 -0.18464175  0.05059647 -0.91706958]
score_train: 0.1365749511987725
score_prediction: -0.02487743179242141
e: 213166.8130206327
